In [ ]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as geemap
except:
    import geemap

# Authenticates and initializes Earth Engine
import ee
ee.Authenticate()

ee.Initialize()

geemap package not installed. Installing ...
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=tAMjn0B4np0HQwkqoi1VMczViFrUhlgLzC2f6sN7184&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/5gEgjW73MpLrYwzYyEAz3q2HO9Kkg0JcRw3k8M8liwtHOCYqQOjGOIk

Successfully saved authorization token.


In [ ]:
#tiles = ["44QLG", "44QLE", "44QLD", "44QKE", "44PKC", "44QKG", "44QKD", "44QMF", "44QKF", "44QME", "44QMD"]
##tiles = ["43QHB", "43QHA", "43QHV", "43QHU", "43PHT", "43QGB", "43QGA", "43QGV", "43QGU"]
##tiles = ["50SKH"]
tiles = ["124034"]
##print(sentinel2_tile_bound)
startDate = ee.Date('2017-01-01')
endDate = ee.Date('2020-09-27')
## function for cloud masking
def maskCloudAndShadows(image):
  ##create function to mask clouds, cloud shadows, snow
  pixel_qa = image.select('pixel_qa');
  ##keep clear (0) and water (1) pixels
  return image.updateMask(pixel_qa.eq(322));   

for tile in tiles:
  tile_bound = ee.FeatureCollection('users/saptarshi/Luancheng_box')
  region = tile_bound.geometry()
  ## Define an example collection.
  ##collection = ee.ImageCollection("LANDSAT/LC08/C01/T2_TOA").filterDate(startDate, endDate).filter(ee.Filter.lt('CLOUD_COVER', 30)).filterBounds(region)
  #collection = ee.ImageCollection('COPERNICUS/S2_SR').filterDate(startDate, endDate).filter(ee.Filter.lt('CLOUD_COVERAGE_ASSESSMENT', 50)).map(maskCloudAndShadows).filter(ee.Filter.eq('MGRS_TILE', tile))
  ##load all Landsat 8 SR image within polygon boundary for last year
  collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(region).filterDate(startDate, endDate).map(maskCloudAndShadows)
  count = int(collection.size().getInfo())
  print("Total number of images: {}".format(count))
  for i in range(0, count):
      image1 = ee.Image(collection.toList(count).get(i))
      dateString = image1.get('system:index').getInfo()
      ndvi = image1.normalizedDifference(['B5','B4'])
      ndvi2 = ndvi.multiply(10000).toInt16()
      #LC08_124034_20170701
      name = dateString[:-15] +"NDVI" + dateString[-16:-4] + "-" + dateString[-4:-2] + "-" + dateString[-2:] # set the output image filename
      #print(name)
      # Set configration parameters for output image
      task_config = {
      ##'folder': 'Sentinel2-NDVI_data', # output Google Drive folder
      'region': region,     # roi 
      'scale': 30,       # image resolution
      'crs': 'EPSG:4326',
      'maxPixels': 1.0E13,
      'fileFormat': 'GeoTIFF'
      }
     
      task = ee.batch.Export.image(ndvi2.clip(region), name, task_config)
      task.start()
      print (name + " is exported")
  print("Process over for tile no: " + tile)

Total number of images: 69
LC08_NDVI_124034_2017-01-22 is exported
LC08_NDVI_124034_2017-02-07 is exported
LC08_NDVI_124034_2017-02-23 is exported
LC08_NDVI_124034_2017-03-11 is exported
LC08_NDVI_124034_2017-03-27 is exported
LC08_NDVI_124034_2017-04-12 is exported
LC08_NDVI_124034_2017-04-28 is exported
LC08_NDVI_124034_2017-05-14 is exported
LC08_NDVI_124034_2017-05-30 is exported
LC08_NDVI_124034_2017-06-15 is exported
LC08_NDVI_124034_2017-07-01 is exported
LC08_NDVI_124034_2017-07-17 is exported
LC08_NDVI_124034_2017-09-19 is exported
LC08_NDVI_124034_2017-10-05 is exported
LC08_NDVI_124034_2017-10-21 is exported
LC08_NDVI_124034_2017-11-06 is exported
LC08_NDVI_124034_2017-11-22 is exported
LC08_NDVI_124034_2017-12-08 is exported
LC08_NDVI_124034_2017-12-24 is exported
LC08_NDVI_124034_2018-01-09 is exported
LC08_NDVI_124034_2018-01-25 is exported
LC08_NDVI_124034_2018-02-10 is exported
LC08_NDVI_124034_2018-02-26 is exported
LC08_NDVI_124034_2018-03-14 is exported
LC08_NDVI_124